### Data Collection


1. GET S&P 500 company info<br>


In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from newsapi import NewsApiClient
from pathlib import Path
%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jakek\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### 1. Get S&P 500 Data

In [3]:
# Get table of the S&P 500 tickers, CIK, and industry from Wikipedia
wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
cik_df = pd.read_html(wiki_url,header=0,index_col=0)[0]
cik_df['GICS Sector'] = cik_df['GICS Sector'].astype("category")
cik_df['GICS Sub Industry'] = cik_df['GICS Sector'].astype("category")
cik_df.tail(100)

,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded,GICS Sub Industry
Symbol,,,,,,,,,
RHI,Robert Half,reports,Industrials,Human Resource & Employment Services,"Menlo Park, California",2000-12-05,315213,1948,Industrials
ROK,Rockwell Automation,reports,Industrials,Electrical Components & Equipment,"Milwaukee, Wisconsin",NaN,1024478,1903,Industrials
ROL,Rollins,reports,Industrials,Environmental & Facilities Services,"Atlanta, Georgia",2018-10-01,84839,1948,Industrials
ROP,Roper,reports,Industrials,Industrial Conglomerates,"Sarasota, Florida",2009-12-23,882835,1981,Industrials
ROST,Ross,reports,Consumer Discretionary,Apparel Retail,"Dublin, California",2009-12-21,745732,1982,Consumer Discretionary
...,...,...,...,...,...,...,...,...,...
YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997,Consumer Discretionary
ZBRA,Zebra,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969,Information Technology
ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927,Health Care


In [4]:
symbol_list = cik_df.index.tolist()
symbol_list

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADM',
 'ADBE',
 'ADP',
 'AAP',
 'AES',
 'AFL',
 'A',
 'AIG',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ANET',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'WRB',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'CHRW',
 'CDNS',
 'CZR',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CERN',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMC

In [49]:
string_1 = ' '.join([symbol_list[i] for i in range(0,50)]).replace(" ", " OR ")
string_2 = ' '.join([symbol_list[i] for i in range(50,100)]).replace(" ", " OR ")
string_3 = ' '.join([symbol_list[i] for i in range(100,150)]).replace(" ", " OR ")
string_4 = ' '.join([symbol_list[i] for i in range(150,200)]).replace(" ", " OR ")
string_5 = ' '.join([symbol_list[i] for i in range(200,250)]).replace(" ", " OR ")
string_6 = ' '.join([symbol_list[i] for i in range(250,300)]).replace(" ", " OR ")
string_7 = ' '.join([symbol_list[i] for i in range(300,350)]).replace(" ", " OR ")
string_8 = ' '.join([symbol_list[i] for i in range(350,400)]).replace(" ", " OR ")
string_9 = ' '.join([symbol_list[i] for i in range(400,450)]).replace(" ", " OR ")
string_10 = ' '.join([symbol_list[i] for i in range(450,500)]).replace(" ", " OR ")


In [5]:
from newsapi import NewsApiClient
load_dotenv()
newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])

headlines = newsapi.get_everything(
    q=,
    language="en",
    page_size=100,
    sort_by="relevancy"
)

In [50]:
headlines_1 = newsapi.get_everything(
    q = string_1,
    language="en",
    page_size=100,
    sort_by="relevancy"
)

headlines_1

In [8]:
# Create the S&p 500 sentiment scores DataFrame
sentiments = []

for article in headlines["articles"]:
    try:
        
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        sentiments.append({
            
            
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu,
            "Text": text
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
df = pd.DataFrame(sentiments)

# Reorder DataFrame columns
cols = ["Compound", "Positive", "Negative", "Neutral","Text"]
df = df[cols]

df.head(10)

,Compound,Positive,Negative,Neutral,Text
0,0.0000,0.000,0.000,1.000,3M CompanyMMM recently announced a 0.7% hike i...
1,0.6369,0.119,0.000,0.881,"Monday, February 14, 2022\r\nThe Zacks Researc..."
2,0.0000,0.000,0.000,1.000,"For Immediate Release\r\nChicago, IL February ..."
3,0.7184,0.167,0.000,0.833,"Making its debut on 05/27/2011, smart beta exc..."
4,0.5574,0.091,0.000,0.909,Wall Street put up a moderate show last week w...
5,0.0000,0.000,0.000,1.000,"For Immediate Release\r\nChicago, IL February ..."
6,0.4939,0.078,0.000,0.922,Its a familiar scene. Youre perched in front o...
7,-0.7764,0.066,0.226,0.708,Cressida Dick absolutely despised Line of Duty...
8,-0.8190,0.041,0.204,0.755,"Mmm, cashmere. So soft and fluffy, right? As w..."
9,0.0000,0.000,0.000,1.000,"For many Russians, it was an unfamiliar sight ..."


In [10]:
df.describe()

,Compound,Positive,Negative,Neutral
count,100.000000,100.000000,100.000000,100.00000
mean,0.152980,0.075890,0.039680,0.88446
std,0.424164,0.080849,0.059685,0.10654
min,-0.819000,0.000000,0.000000,0.60400
25%,0.000000,0.000000,0.000000,0.80375
50%,0.000000,0.064000,0.000000,0.90800
75%,0.510600,0.132000,0.068250,1.00000
max,0.910000,0.314000,0.234000,1.00000


In [11]:
# Instantiate the lemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Expand the default stopwords list if necessary
sw = set(stopwords.words('english'))

In [ ]:
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text
    
    regex = re.compile("[^a-zA-Z ]")
   
    # Create a tokenized list of the words
    re_clean = regex.sub('', text)
    words = word_tokenize(re_clean)
    words = word_tokenize(re_clean)
    words = word_tokenize(re_clean.lower())
    
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
   
    # Convert the words to lowercase
    words = [word for word in words if word not in sw]
    
    # Remove the stop words
   
    return lem

tokenizer(df.iloc[0]['Text'])

In [ ]:
df['tokens'] = df['Text'].apply(tokenizer)
df

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
def get_token(df):
    tokens = []
    for i in df['tokens']:
        tokens.extend(i)
    return tokens

tokens = get_token(df)


In [ ]:
def bigram_counter(tokens, N):
    words_count = dict(Counter(ngrams(tokens, n=2)))
    return words_count

In [ ]:
# Function token_count generates the top 10 words for a given stock
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [ ]:
token_count(tokens, 10)

In [ ]:
import spacy
from spacy import displacy

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
text = ' '.join(df['Text'])
text

In [ ]:
doc = nlp(text)

In [ ]:
# Add a title to the document
doc.user_data["title"] = "NER"

In [ ]:
# Render the visualization
displacy.render(doc, style='ent')

In [ ]:
# List all Entities
org_list = []

for ent in doc.ents:
    if ent.label_ == 'ORG':
        org_list.append(ent.text)

org_list = Counter(org_list).most_common(100)

df_org = pd.DataFrame(org_list, columns = ['text', 'count'])

df_org.head(50)

In [ ]:
# Dropping Reuters as it is a publisher
df_org = df_org.iloc[2:]
df_org

In [ ]:
headlines = newsapi.get_everything(
    q="Polen" "",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

In [ ]:
# Create the Polen sentiment scores DataFrame
sentiments = []

for article in headlines["articles"]:
    try:
        
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        sentiments.append({
            
            
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu,
            "Text": text
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
df_polen = pd.DataFrame(sentiments)

# Reorder DataFrame columns
cols = ["Compound", "Positive", "Negative", "Neutral","Text"]
df_polen = df[cols]

df_polen.head(10)

In [ ]:
df_polen.describe()